In [1]:
import pandas as pd
import numpy as np
# from pivottablejs import pivot_ui
# import qgrid

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
pd.set_option("mode.chained_assignment", None)
import plotly.express as px
import tkinter as tk
from tkinter import filedialog
from datetime import datetime, date
import pyodbc
import time
import timeit
import sqlalchemy
import dateutil.relativedelta
import warnings
warnings.filterwarnings("ignore")
from PIL import Image, ImageTk

import os

## SQL數據處理class
- 需要寫兩個.py檔，1個是初始化數據庫使用，1個是用在日常更新使用，更新頻率為半小時

In [ ]:
# class for initialization
class SQL_data_init:
    
    last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
    
    def __init__(self, init=True) -> None:
        self._server = "10.226.0.53"
        self._database = "master"
        self._username = "BIUser"
        self._password = "test"
        self._cnxn = self.conn_server()
        self._server_ex = "10.226.61.50"
        self._database_ex = "SAP_DW"
        self._cnxn_ex = self.conn_server(fetch=False)
        self.init = init
        self.df = self.fetch_new_data(init=init)
        self.clean_df = self.data_preprocessing()
        self.loading2view()
        
            
    
    def conn_server(self, fetch=True):
        if fetch == True:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server,database=self._database ,uid=self._username,password=self._password)
        else:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server_ex,database=self._database_ex ,uid=self._username,password=self._password)
        return cnxn
    
    
    def fetch_new_data(self, init=True):
        if init == True:
            '''When init=False, initialize the data'''
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230101'"
        else:
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(self.last_month)
        df = pd.read_sql(script, self._cnxn)
        return df
    
    
    def data_preprocessing(self):
        df = self.df
        df["CreatedDateTime"] = datetime.now()
        df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']].fillna(0)
        return df
        
    
    
    def loading2view(self):
        df = self.clean_df
        conn = self._cnxn_ex
        cursor = conn.cursor()
        if self.init == True:
            cursor.execute("DELETE from ZMAW")
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        else:
            delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(self.last_month)
            cursor.execute(delete)
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        conn.commit()
        cursor.close()
        
        
if __name__ == "__main__":
    SQL_data_init()
        

In [ ]:
# Routine update
class SQL_data:
    
    last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
    
    def __init__(self, init=False) -> None:
        self._server = "10.226.0.53"
        self._database = "master"
        self._username = "BIUser"
        self._password = "test"
        self._cnxn = self.conn_server()
        self._server_ex = "10.226.61.50"
        self._database_ex = "SAP_DW"
        self._cnxn_ex = self.conn_server(fetch=False)
        self.init = init
        self.df = self.fetch_new_data(init=init)
        self.clean_df = self.data_preprocessing()
        self.loading2view()
        
            
    
    def conn_server(self, fetch=True):
        if fetch == True:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server,database=self._database ,uid=self._username,password=self._password)
        else:
            cnxn = pyodbc.connect(driver='SQL Server',server=self._server_ex,database=self._database_ex ,uid=self._username,password=self._password)
        return cnxn
    
    
    def fetch_new_data(self, init=True):
        if init == True:
            '''When init=False, initialize the data'''
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230201'"
        else:
            script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(self.last_month)
        df = pd.read_sql(script, self._cnxn)
        return df
    
    
    def data_preprocessing(self):
        df = self.df
        df["CreatedDateTime"] = datetime.now()
        df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
        '未稅銷貨']].fillna(0)
        return df
        
    
    
    def loading2view(self):
        df = self.clean_df
        conn = self._cnxn_ex
        cursor = conn.cursor()
        if self.init == True:
            cursor.execute("DELETE from ZMAW")
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        else:
            delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(self.last_month)
            cursor.execute(delete)
            cursor.commit()
            for index, row in df.iterrows():
                cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員簡稱, 大胎業務員簡稱, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
        conn.commit()
        cursor.close()
        
        
if __name__ == "__main__":
    SQL_data()
        

In [25]:
# Verify the result
server = "10.226.0.53"
# server = "10.226.0.53"
database = "SAP_DW"
username = "BI_User"
password = "BsSQLDB"

conn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
cursor = conn.cursor()
sql = "select * from [dbo].[V_ZMAW] where 中計 like '%TBR%' OR 中計 like '%LSR2%' and 訂單類型 in ('ZSO1', 'ZKE', 'ZOR1') and 配銷通路 in ('OT', 'FT', 'AG')"
df = pd.read_sql(sql=sql, con=conn)
df

,組織,出貨點,所名,區域,訂單建立日,訂單文件日,訂單建立時,交貨文件日,計劃出貨日,實際出貨日,交單建立時,請款建立日,請款日期,請款建立時,訂單建立人,交單建立人,請款建立人,訂單備註,產品階層,訂單類型,訂單類型說明,交貨類型,交貨類型說明,請款類型,請款類型說明,訂單單號,訂單項次,交貨單號,交貨項次,請款文件,請款文件項次,發票號碼,買方,送貨方,彙總客戶,業務員,小胎業務員,大胎業務員,買方全名,買方簡稱,送貨方全名,送貨方簡稱,彙總客戶全名,彙總客戶簡稱,業務員全名,業務員簡稱,小胎業務員全名,大胎業務員全名,配銷通路,販社通路,大胎契約,小胎契約,付款條件,物料群組,分類,物料,物料說明,花紋別,Premium,吋別,中計,訂單數量,交貨數量,發票單價,發票未稅,稅額,發票合計,銷售數量,銷貨單價,未稅銷貨,M物料說明,物料類型,物料類型說明,M物料群組,M花紋別,花紋類別,花紋分類,花紋,新物料號碼,大小尺寸,一般項目類別群組,胎寬,扁平比,CONST,M吋別,速度荷重比,層數,基本構造,構造材質1,構造材質2,構造材質3,特殊構造,表示1,表示2,表示3,表示4,中計商品代號1,中計商品代號2,BBGF,系列,規格_REP,花紋_REP,合併料號,PSRG分類,TBRG分類,交貨數量統計,大分類,中分類,小分類,配銷通路分類,販社通路分類
0,FCC1,FF73,高雄所,南區,2020-06-30,2020-06-30,144059,2020-06-30,2020-06-30,2020-06-30,144115,2020-06-30,2020-06-30,190811,F51SAS01,F51SAS01,F51SAS01,None,,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103817983,000010,0280050924,000010,0090753342,000032,CJ87024580,L1L1G033,L1L1G033,L1L1G033,U01,U23,U02,安速輪胎有限公司,安速,安速輪胎有限公司,安速,安速輪胎有限公司,安速,李惠昌,李惠昌,張廷豪,張正平,OT,H,BFP,鑽石店,W030,TLSRJ,進口,LSR03733,215/75R175 124L R294Z,R294Z,None,175,LSR2,4.0,4.0,6239.0,23768.0,1188.0,24956.0,4.0,6239.0,23768.0,215/75R175 124L R294Z,TER1,成品-TIRE,TLSRJ,R294Z,None,None,R294,,175,ZORM,215,75,R,175,124L,,T,KC,K1,K3,,,,EF,,LR2,LSR2,,,215/75R17.5,R294,G0578,X,LSR2,4.0,TBRG,LSR2,LSR2-Non PREMIUM,REP,Other
1,FCC1,FF73,高雄所,南區,2020-07-14,2020-07-14,114815,2020-07-14,2020-07-14,2020-07-14,114906,2020-07-31,2020-07-31,191843,F51SAS01,F51SAS01,F51SAS01,第一班車下午,,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103828170,000010,0280061586,000010,0090756112,000038,EE02118839,L1L1G033,L1L1G033,L1L1G033,U01,U23,U02,安速輪胎有限公司,安速,安速輪胎有限公司,安速,安速輪胎有限公司,安速,李惠昌,李惠昌,張廷豪,張正平,OT,H,BFP,鑽石店,W030,TLSRJ,進口,LSR03733,215/75R175 124L R294Z,R294Z,None,175,LSR2,6.0,6.0,6239.0,35651.0,1783.0,37434.0,6.0,6239.0,35651.0,215/75R175 124L R294Z,TER1,成品-TIRE,TLSRJ,R294Z,None,None,R294,,175,ZORM,215,75,R,175,124L,,T,KC,K1,K3,,,,EF,,LR2,LSR2,,,215/75R17.5,R294,G0578,X,LSR2,6.0,TBRG,LSR2,LSR2-Non PREMIUM,REP,Other
2,FCC1,FF63,員林所,中區,2020-07-23,2020-07-23,100301,2020-07-23,2020-07-23,2020-07-23,100310,2020-07-31,2020-07-31,135115,F32SAS01,F32SAS01,F32SAS01,None,,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103835026,000010,0280069086,000010,0090755012,000013,EE02100890,I1I1B010,I1I1B010,I1I1B010,R05,O07,O26,信輪輪胎行,信輪,信輪輪胎行,信輪,信輪輪胎行,信輪,詹期友,詹期友,李健偉,黃啟禮,OT,H,一般店,鑽石店,W030,TLSRJ,進口,LSR03733,215/75R175 124L R294Z,R294Z,None,175,LSR2,2.0,2.0,6239.0,11884.0,594.0,12478.0,2.0,6239.0,11884.0,215/75R175 124L R294Z,TER1,成品-TIRE,TLSRJ,R294Z,None,None,R294,,175,ZORM,215,75,R,175,124L,,T,KC,K1,K3,,,,EF,,LR2,LSR2,,,215/75R17.5,R294,G0578,X,LSR2,2.0,TBRG,LSR2,LSR2-Non PREMIUM,REP,Other
3,FCC1,FF73,高雄所,南區,2023-06-27,2023-06-27,111655,2023-06-27,2023-06-27,2023-06-27,111658,2023-07-01,2023-06-30,131409,F41SAS02,F41SAS02,F51SAS02,None,NET,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0104516753,000010,0280803303,000010,0090823435,000011,PJ25362169,K1K10029,K1K10029,K1K10029,U24,U24,T04,登昌輪胎行,登昌,登昌輪胎行,登昌,登昌輪胎行,登昌,李彥興,,李彥興,張鴻宗,OT,H,一般店,一般店,W030,TLSRJ,進口,LSR03733,215/75R175 124L R294Z,R294Z,None,175,LSR2,2.0,2.0,6615.0,12600.0,630.0,13230.0,2.0,6615.0,12600.0,215/75R175 124L R294Z,TER1,成品-TIRE,TLSRJ,R294Z,None,None,R294,,175,ZORM,215,75,R,175,124L,,T,KC,K1,K3,,,,EF,,LR2,LSR2,,,215/75R17.5,R294,G0578,X,LSR2,2.0,TBRG,LSR2,LSR2-Non PREMIUM,REP,Other
4,FCC1,FF13,桃園所,北區,2021-08-04,2021-08-04,093834,2021-08-04,2021-08-04,2021-08-04,093856,2021-08-31,2021-08-31,230617,F42SAS02,F42SAS02,F12SAS02,送詠靖_94846_080請拿發票與報修單,,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0104090757,000010,0280345044,000010,0090782007,000011,RC84331694,E1E10113,E1E10113,E1E10113,E11,E11,D08,文鹿汽車有限公司,文鹿汽車,文鹿汽車有限公司,文鹿汽車,文鹿汽車有限公司,文鹿汽車,陳家興,陳家興,陳家興,林俊良,OT,H,BFP,競品店,Z030,TLSRJ,進口,LSR01270,825 R 16 14P R225Z,R225Z,None,16,LSR2,1.0,1.0,6050.0,5762.0,288.0,6050.0,1.0,6050.0,5762.0,825 R 16 14P R225Z,TER1,成品-TIRE,TLSRJ,R225Z,None,None,R225,,16,ZORM,825,,R,16,,14,,KC,K1,K3,,,,D0,,LR2,LSR2,,,825R16,R225,G0883,X,LSR2,1.0,TBRG,LSR2,LSR2-Non PREMIUM,REP,Other
...,...,...,...,...,...,...,...,...,..

In [12]:
# 計算業績販社只有ZSO1和ZKE的訂單類型有效，本社是ZOR1（TBR(G)）
result = df[(df["中計"].str.contains("TBR|LSR2")) & (df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["配銷通路"].isin(["OT", "FT", "AG"]))]
result["銷售數量"].sum()
result
# result["買方"].unique()



,組織,出貨點,所名,區域,訂單建立日,訂單文件日,訂單建立時,交貨文件日,計劃出貨日,實際出貨日,交單建立時,請款建立日,請款日期,請款建立時,訂單建立人,交單建立人,請款建立人,訂單備註,產品階層,訂單類型,訂單類型說明,交貨類型,交貨類型說明,請款類型,請款類型說明,訂單單號,訂單項次,交貨單號,交貨項次,請款文件,請款文件項次,發票號碼,買方,送貨方,彙總客戶,業務員,小胎業務員,大胎業務員,買方全名,買方簡稱,送貨方全名,送貨方簡稱,彙總客戶全名,彙總客戶簡稱,業務員全名,業務員簡稱,小胎業務員全名,大胎業務員全名,配銷通路,販社通路,大胎契約,小胎契約,付款條件,物料群組,分類,物料,物料說明,花紋別,Premium,吋別,中計,訂單數量,交貨數量,發票單價,發票未稅,稅額,發票合計,銷售數量,銷貨單價,未稅銷貨,CreatedDateTime
46,BSFC,FF06,彰濱成品庫,None,2019-12-30,2019-12-30,113336,2020-01-02,2020-01-02,2020-01-02,093852,2020-01-22,2020-01-22,113738,FHQCMS01,F06WHS01,FHQ10S06,1/2到貨 抄胎號,,ZOR1,本社 - 一般訂單,ZF01,本社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103694606,000030,0083918415,000010,0090741699,000030,YP69907753,BC112014,BC112014,BC112014,B00,None,None,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,BSFC業務員,BSFC業務員,None,None,FT,Z,Z,Z,Z090,TTBR,進口,TBR05258,255/70R225 B40M R294Z,R294Z,None,225,TBR3,4.0,4.0,9293.0,35400.0,1770.0,37170.0,4.0,9293.0,35400.0,2023-04-14 18:11:09.387
47,BSFC,FF06,彰濱成品庫,None,2019-12-30,2019-12-30,113336,2020-01-02,2020-01-02,2020-01-02,093852,2020-01-22,2020-01-22,113738,FHQCMS01,F06WHS01,FHQ10S06,1/2到貨 抄胎號,,ZOR1,本社 - 一般訂單,ZF01,本社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103694606,000010,0083918415,000020,0090741699,000031,YP69907753,BC112014,BC112014,BC112014,B00,None,None,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,BSFC業務員,BSFC業務員,None,None,FT,Z,Z,Z,Z090,TTBR,進口,TBR0J295,295/80R225 152M R156AZ,R156AZ,None,225,TBR3,80.0,80.0,10080.0,768000.0,38400.0,806400.0,80.0,10080.0,768000.0,2023-04-14 18:11:09.387
48,BSFC,FF06,彰濱成品庫,None,2019-12-30,2019-12-30,113336,2020-01-02,2020-01-02,2020-01-02,093852,2020-01-22,2020-01-22,113738,FHQCMS01,F06WHS01,FHQ10S06,1/2到貨 抄胎號,,ZOR1,本社 - 一般訂單,ZF01,本社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103694606,000020,0083918415,000030,0090741699,000032,YP69907753,BC112014,BC112014,BC112014,B00,None,None,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯客運,BSFC業務員,BSFC業務員,None,None,FT,Z,Z,Z,Z090,TTBR,進口,TBR0J458,275/70R22.5 148J R162AZ,R162AZ,None,225,TBR3,10.0,10.0,9870.0,94000.0,4700.0,98700.0,10.0,9870.0,94000.0,2023-04-14 18:11:09.390
49,BSFC,FF06,彰濱成品庫,None,2019-12-30,2019-12-30,112727,2020-01-02,2020-01-02,2020-01-02,093852,2020-01-08,2020-01-08,172631,FHQCMS01,F06WHS01,FHQ10S06,1/2到貨 抄胎號,,ZOR1,本社 - 一般訂單,ZF01,本社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103694581,000010,0083918416,000010,0090741358,000010,YP69907724,BC108041,0010804105,BC108041,B00,None,None,龍福國際股份有限公司,龍福國際,龍福國際股份有限公司,龍福國際,龍福國際股份有限公司,龍福國際,BSFC業務員,BSFC業務員,None,None,FT,Z,Z,Z,Z100,TTBR,進口,TBR0J247,295/80R225 152M R150Z,R150Z,None,225,TBR3,20.0,20.0,10080.0,192000.0,9600.0,201600.0,20.0,10080.0,192000.0,2023-04-14 18:11:09.390
50,BSFC,FF06,彰濱成品庫,None,2019-12-30,2019-12-30,112038,2020-01-02,2020-01-02,2020-01-02,102909,2020-01-22,2020-01-22,113738,FHQCMS01,F06WHS01,FHQ10S06,1/3到貨 抄胎號,,ZOR1,本社 - 一般訂單,ZF01,本社 - 標準訂單交貨,ZF2,發票_一般銷貨發票,0103694545,000010,0083918586,000010,0090741699,000033,YP69907753,BC112014,0011201416,BC112014,B00,None,None,統聯汽車客運股份有限公司,統聯客運,統聯汽車客運股份有限公司,統聯,統聯汽車客運股份有限公司,統聯客運,BSFC業務員,BSFC業務員,None,None,FT,Z,Z,Z,Z090,TTBR,進口,TBR05258,255/70R225 B40M R294Z,R294Z,None,225,TBR3,8.0,8.0,9293.0,70800.0,3540.0,74340.0,8.0,9293.0,70800.0,2023-04-14 18:11:09.390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361481,FCC1,FF73,高雄所,南區,2023-07-14,2023-07-14,094221,2023-07-14,2023-07-14,2023-07-14,094239,None,None,None,F41SAS01,F41SAS01,None,None,,ZSO1,販社 - 一般訂單,ZF02,販社 - 標準訂單交貨,None,None,0104527106,000010,0280814471,000010,None,None,None,K1K10012,K1K10012,K1K10012,T05,T05,T04,新泓源輪胎行,新泓源,新泓源輪胎行,新泓源,新泓源輪胎行,新泓源,林純弘,林純弘,林純弘,張鴻宗,OT,H,None,None,None,TLSRJ,進口,LSR01270,825 R 16 14P R225Z,R225Z,None,16,LSR2,5.0,5.0,NaN,NaN,NaN,NaN,5.0,6948.0,33085.0,2023-07-14 11:47:36.713
1361524,FCC1,FF11,鶯歌所,北區,2023-07-14,2023-07-14,094843,2023-07-14,2023-07-14,2023-07-14,100532,None,None,None,F11SAS

## SQL Server 連線

In [2]:
server = "10.226.0.53"
database = "SAP_DW"
username = "BI_User"
password = "BsSQLDB"

In [ ]:
cnxn = pyodbc.connect(driver='SQL Server', server=server, database=database, uid=username, password=password)
pd.read_sql("select * from ZMAW", cnxn)

In [2]:
server = "10.226.0.53"
database = "master"
username = "BI_User"
password = "BsSQLDB"

In [3]:
cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)

In [ ]:
cursor = cnxn.cursor()

In [ ]:
last_month = "'" + (datetime.now() + dateutil.relativedelta.relativedelta(months=-1)).strftime("%Y%m") + "01'"
last_month

In [ ]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= {}".format(last_month)

In [4]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where 實際出貨日 >= '20230301'"

In [5]:
df = pd.read_sql(script, cnxn)

In [6]:
df = df[(df["中計"].str.contains("TBR|LSR2")) & (df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["配銷通路"].isin(["OT", "FT", "AG"]))]
df["交貨數量"].sum()

3751.0

In [ ]:
df

In [ ]:
df[df["配銷通路"].isna()]

In [ ]:
inventory_status = df[df["料號"].str.contains("TBR|LSR")]

In [ ]:
inventory_status[inventory_status["未限制庫存"] != 0]["倉別名稱"].value_counts()

In [ ]:
# 查看系統所有訂單的script
current_mon = datetime.now().strftime("'%Y-%m'")
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = {}".format(current_mon)

In [ ]:
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ZMAW where convert(varchar(7), 訂單建立日, 120) >= '2023-01'"

In [ ]:
## 所有訂單（包含販社和本社）
category_table = pd.read_excel("./ReferenceTable/20220801_category_types.xlsx")
cuscode_type = dict(zip(category_table["客戶代號"], category_table["大通路"]))
df["大通路"] = df["買方"].map(cuscode_type)

In [ ]:
cursor.close()

In [ ]:
df.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\sql_sever_data_from_2019ytd.pkl")

In [ ]:
df = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\sql_sever_data_from_2019ytd.pkl")

## 測試所有訂單的想法

In [ ]:
server = "10.226.0.53"
database = "master"
username = "BIUser"
password = "test"

cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = '2023-02'"

In [ ]:
df = pd.read_sql(script, cnxn)

In [ ]:
df["大通路"] = df["彙總客戶"].map(cuscode_type)

In [ ]:
df = df[(df["訂單類型"].isin(["ZSO1", "ZKE", "ZOR1"])) & (df["大通路"].notna()) & (df["大通路"] == "BSSC") & (df["物料說明"].str.contains("315/80R225")) & (df["花紋別"].str.contains("R150|M729"))]

In [ ]:
df.groupby(["買方全名", "業務員全名", "物料說明", "花紋別"])[["訂單數量"]].sum().reset_index().to_excel(r"C:\Users\kc.hsu\Desktop\2月特別販促清單.xlsx", index=False)

In [ ]:
df.groupby(["大通路"])[["訂單數量"]].sum()

In [ ]:
logo = Image.open(r"D:\kc.hsu\My Documents\My Pictures\BS方形logo.png")

In [ ]:
logo.save(r"D:\kc.hsu\My Documents\My Pictures\BS方形logo.ico", format='ICO')

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\2021-2022_Size Data_All_TBR_rawling_data.xlsx", sheet_name="Sheet1")

In [ ]:
ob = df[(df["預算/前實績"] == "23OB_V2") & (df["國內市場財別"] == "TBR(G)")]

In [ ]:
ob_target = int(ob[(ob["年月"] == int(datetime.now().strftime("%Y%m"))) & (ob["通路明細"] == "FLT-REP")]["條數"].sum())

In [ ]:
class RealTimeAct(tk.Tk):
    server = "10.226.0.53"
    database = "master"
    username = "BIUser"
    password = "test"

    cnxn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)
    script = "select * from [10.212.27.198].FC0.dbo.V_BI_ORDER where convert(varchar(7), 訂單建立日, 120) = '2023-01'"
    
    def __init__(self):
        super().__init__()
        self.wm_attributes("-topmost", 1)
        self.wm_title("每日TBR(G)販社訂單即時數據")
        # self.wm_maxsize(400, 150)
        self.connection = pd.read_sql(self.script, self.cnxn)
        # self.parent = parent
        
        self.refresh_data()
        # self.run()
        self.mainloop()
        
    def refresh_data(self):
        df = self.connection
        df["中計"] = df["中計"].fillna("")
        tbrg = df[df["中計"].str.contains("LSR2|TBR")]
        result = tbrg[tbrg["訂單類型"].isin(["ZKE", "ZSO1"])][["訂單數量", "交貨數量"]].sum()
        orderLabel = tk.Label(self, text="目前販社訂單總數: {}".format(int(result[0])), font=("Arial", 20)).grid(row=0, column=0)
        shipLabel = tk.Label(self, text="目前販社出貨總數: {}".format(int(result[1])), font=("Arial", 20)).grid(row=1, column=0)
        timeLabel = tk.Label(self, text="更新時間：{}".format(time.strftime("%Y-%m-%d %H:%M")), font=("Arial", 12)).grid(row=2, column=0)
        self.after(20000, self.refresh_data)
        # return result
    
    
    def run(self):
        # orderLabel = (self, text="目前販社訂單總數: {}".format(int(self.refresh_data[0])), font=("Arial", 20)).grid(row=0, column=0)
        # shipLabel = self.Label(self, text="目前販社出貨總數: {}".format(int(self.refresh_data[1])), font=("Arial", 20)).grid(row=1, column=0)
        # timeLabel = self.Label(self, text="更新時間：{}".format(time.strftime("%Y-%m-%d %H:%M")), font=("Arial", 12)).grid(row=2, column=0)
        pass
        
    

In [ ]:
RealTimeAct()

In [ ]:
while time.strftime("%H:%M", time.localtime()) != "17:00":
    df = pd.read_sql(script, cnxn)
    df["中計"] = df["中計"].fillna("")
    tbrg = df[df["中計"].str.contains("LSR2|TBR")]
    result = tbrg[tbrg["訂單類型"].isin(["ZKE", "ZSO1"])][["訂單數量", "交貨數量"]].sum()
    print(result)
    if result[1] + 1021 >= 9052:
        print("恭喜達標")
        break
    else:
        continue

## 使用BPCOM的數據來帶入Product name

In [ ]:
bpcom = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\產品規格相關\BPCOM\BPCOM_202208040543.xlsx")

In [ ]:
bpcom["Product Name"].map(lambda x: x.split(" ")[0]).value_counts()

In [ ]:
check_dict = dict(zip(bpcom["PrCode"], bpcom["BP"]))

In [ ]:
df["花紋別"] = df["物料"].map(check_dict)

In [ ]:
result = df[df["中計"].str.contains("TBR|LSR2")]

## 測試寫入SQL Server

In [2]:
server = "10.226.0.53"
database = "SAP_DW"
username = "BI_User"
password = "BsSQLDB"

In [3]:
conn = pyodbc.connect(driver='SQL Server',server=server,database=database ,uid=username,password=password)

In [5]:
cursor = conn.cursor()

In [5]:
inventory = pd.read_sql("SELECT * from [10.212.27.198].FC0.dbo.V_BI_INVENTORY WHERE Subcode='REP' ORDER BY 倉別代號", con=conn)

In [6]:
inventory

,料號,物料說明,吋別,花紋別,中計,組織,倉別代號,倉別名稱,Subcode,可用庫存
0,PSR0F930,195/65R15 091H EP15KZ,15,EP15KZ,PSR4,BSFC,FF01,新工成品庫,REP,0.0
1,PSR0F977,205/55R16 091V EP15UZ,16,EP15UZ,PSR3,BSFC,FF01,新工成品庫,REP,1.0
2,PSR0FA26,215/60R17 096H AL01LZ,17,AL01LZ,PSR6,BSFC,FF01,新工成品庫,REP,1.0
3,PSR0FA33,215/60R17 096H AL01LZ,17,AL01LZ,PSR6,BSFC,FF01,新工成品庫,REP,4.0
4,PSR0FA34,225/50R18 095V AL01KZ,18,AL01KZ,PSR6,BSFC,FF01,新工成品庫,REP,3.0
...,...,...,...,...,...,...,...,...,...,...
36203,PSR15999,215/60R17 096H T05AAZ,17,T05AAZ,PSR2,FCC1,FF76,台南凍,REP,0.0
36204,PSR16181,215/45R18 089W T05ACZ,18,T05ACZ,PSR2,FCC1,FF76,台南凍,REP,13.0
36205,PSR17323,104V P265/45R21 D400GZ,21,G400GZ,PSR2,FCC1,FF76,台南凍,REP,6.0
36206,TBR0J067,12R225 16P 152L R150AZ,225,R150AZ,TBR3,FCC1,FF76,台南凍,REP,0.0


In [ ]:
delete = "DELETE from ZMAW where 實際出貨日 >= {}".format(last_month)

In [ ]:
delete = "DELETE from ZMAW where 實際出貨日 >= '20221226'"

In [ ]:
cursor.execute("DELETE from zmaw")

In [ ]:
cursor.commit()

In [ ]:
df = pd.read_sql("select * from [dbo].[ZMAW]", con=conn)
df

In [ ]:
df["CreatedDateTime"] = datetime.now()

In [ ]:
df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
       '未稅銷貨']] = df[['訂單數量', '交貨數量', '發票單價', '發票未稅', '稅額', '發票合計', '銷售數量', '銷貨單價',
       '未稅銷貨']].fillna(0)

In [ ]:
df.columns

In [ ]:
for index, row in df.iterrows():
    cursor.execute('INSERT INTO ZMAW (組織, 出貨點, 所名, 區域, 訂單建立日, 訂單文件日, 訂單建立時, 交貨文件日, 計劃出貨日, 實際出貨日, 交單建立時, 請款建立日, 請款日期, 請款建立時, 訂單建立人, 交單建立人, 請款建立人, 訂單備註, 產品階層, 訂單類型, 訂單類型說明, 交貨類型, 交貨類型說明, 請款類型, 請款類型說明, 訂單單號, 訂單項次, 交貨單號, 交貨項次, 請款文件, 請款文件項次, 發票號碼, 買方, 送貨方, 彙總客戶, 業務員, 小胎業務員, 大胎業務員, 買方全名, 買方簡稱, 送貨方全名, 送貨方簡稱, 彙總客戶全名, 彙總客戶簡稱, 業務員全名, 業務員簡稱, 小胎業務員, 大胎業務員, 配銷通路, 販社通路, 大胎契約, 小胎契約, 付款條件, 物料群組, 分類, 物料, 物料說明, 花紋別, Premium, 吋別, 中計, 訂單數量, 交貨數量, 發票單價, 發票未稅, 稅額, 發票合計, 銷售數量, 銷貨單價, 未稅銷貨, CreatedDateTime) values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', tuple(row))
    # print(tuple(row))

In [ ]:
conn.commit()

In [ ]:
cursor.close()

## 數據轉換function

In [2]:
cur_month = datetime.today().date().strftime("%Y-%m") + "-01"

In [3]:
class DataExtract:
    
    type_keys = ['訂單日期', '實際出貨日', '訂單單號', '訂單項次', '買方', '物料', '物料說明', '國別', '銷售數量', '銷貨單價']
    type_values = [str, str, str, str, str, str, str, str, int, int]
    type_dict = dict(zip(type_keys, type_values))
    
    def __init__(self):
        self.path = self.get_path()
        
    def get_path(self):
        paths = []
        while True:
            answer = input("選擇路徑?(Y/N)").upper()
            if answer == "Y":
                root = tk.Tk()
                root.withdraw()
                file_path = filedialog.askopenfilenames()
                paths += file_path    
            else:
                break
        return paths
            
        
    def read_data(self):
        dfs = []
        # path = self.path.replace("\\", "/")
        for path in self.path:
            if path.split(".")[-1] == "csv" and "ag" in path:
                ag = pd.read_csv(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            elif path.split(".")[-1] == "pkl" and "ag" in path:
                ag = pd.read_pickle(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            elif path.split(".")[-1] == "csv" and "sc" in path:
                sc = pd.read_csv(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
            elif path.split(".")[-1] == "pkl" and "sc" in path:
                sc = pd.read_pickle(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
            elif path.split(".")[-1] == "xlsx" and "ag" in path:
                ag = pd.read_excel(path)
                ag = self.ag_extract(ag)
                dfs.append(ag)
            else:
                sc = pd.read_excel(path)
                sc = self.sc_extract(sc)
                dfs.append(sc)
        data = pd.concat(dfs)
        data = data.drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last")
        return data

    def ag_extract(self, df):
        # ag = self.read_data()
        ag = df
        ag.columns = ag.columns.str.strip()
        ag = ag[(ag["S_配銷通路"].isin(["AG", "FT"])) & (ag["S_中計商品代號1"].isin(["LR2", "TR2", "TR3", "TR5", "TR4"]))]
        ag = ag[["D_文件日期", "D_實際發貨日期", "S_訂單單號", "S_項次", "S_買方(客戶號碼)", "S_物料編號", "S_物料說明", "S_國別", "S_數量", "S_項目淨值"]]
        ag.columns = self.type_keys
        # ag = ag.astype(self.type_dict)
        return ag
    
    def sc_extract(self, df):
        # sc = self.read_data()
        sc = df
        sc.columns = sc.columns.str.strip()
        sc = sc[sc["中計"].isin(["LSR2", "TBR3", "TBR5", "TBR2", "TBR4"])]
        sc = sc[["訂單建立日", "實際出貨日", "訂單單號", "訂單項次", "買方", "物料", "物料說明", "國別", "銷售數量", "銷貨單價"]]
        sc.columns = self.type_keys
        # sc = sc.astype(self.type_dict)
        return sc
    
    def sales_preprocessing(self):
        df = self.read_data()
        df["訂單日期"] = df["訂單日期"].dt.date
        df["實際出貨日"] = df["實際出貨日"].dt.date
        df = df.astype(self.type_dict)
        df["訂單單號"] = df["訂單單號"].map(lambda x: x.replace(".0", "") if "." in x else x)
        df["訂單項次"] = df["訂單項次"].map(lambda x: x.replace(".0", "") if "." in x else x)
        df["銷貨單價"] = df["銷貨單價"].fillna(0).astype(int)
        return df

In [11]:
result = DataExtract().sales_preprocessing()

In [12]:
### 開啟客戶對照表
# 開啟客編客戶對照表
# category_table = pd.read_csv("./ReferenceTable/category_types.csv")
category_table = pd.read_excel("./ReferenceTable/20220801_category_types.xlsx")
# cat_table = qgrid.show_grid(category_table, show_toolbar=True)
# cat_table

# 如果有修改的話，修改完可以用.get_changed_df()來獲得修改過後的表格

In [13]:
cuscode_cus = dict(zip(category_table["客戶代號"], category_table["合併客戶名稱"]))
cuscode_type = dict(zip(category_table["客戶代號"], category_table["大通路"]))
cuscode_channel = dict(zip(category_table["客戶代號"], category_table["小通路"]))
sales_dict = dict(zip(category_table["客戶代號"], category_table["業務名稱"]))
sales_area = dict(zip(category_table["客戶代號"], category_table["負責區域"]))
sales_branch = dict(zip(category_table["客戶代號"], category_table["所屬區域"]))
cus_address = dict(zip(category_table["客戶代號"], category_table["住址"]))
cus_city = dict(zip(category_table["客戶代號"], category_table["城市"]))
rep_dict = dict(zip(category_table["客戶代號"], category_table["母通路"]))
liwei_dict = dict(zip(category_table["客戶代號"], category_table["liwei"]))
ranking = dict(zip(category_table["客戶代號"], category_table["Ranking"]))
cus_area = dict(zip(category_table["客戶代號"], category_table["客戶所在地分區"]))

In [14]:
pcode = pd.read_excel("./ReferenceTable/20220721_PCODE整理表.xlsx")
size_dict = dict(zip(pcode["Old PCODEs"], pcode["size"]))
pattern_dict = dict(zip(pcode["Old PCODEs"], pcode["pattern"]))
origin_dict = dict(zip(pcode["Old PCODEs"], pcode["p_origin"]))
tbr_prem = dict(zip(pcode["Old PCODEs"], pcode["TBR Premium"]))
group = dict(zip(pcode["Old PCODEs"], pcode["group"]))

In [15]:
result.reset_index(drop=True).to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\rolling_sales_data.pkl")
long = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\rolling_sales_data.pkl")

In [16]:
long["客戶名稱"] = long["買方"].map(cuscode_cus)
long["母通路"] = long["買方"].map(rep_dict)
long["大通路"] = long["買方"].map(cuscode_type)
long["小通路"] = long["買方"].map(cuscode_channel)
long["日報通路"] = long["買方"].map(liwei_dict)
long["業務名稱"] = long["買方"].map(sales_dict)
long["業務負責區域"] = long["買方"].map(sales_area)
long["業務所屬區域"] = long["買方"].map(sales_branch)
long["客戶登記地址"] = long["買方"].map(cus_address)
long["客戶城市"] = long["買方"].map(cus_city)
long["客戶等級"] = long["買方"].map(ranking)
long["規格"] = long["物料"].map(size_dict)
long["花紋"] = long["物料"].map(pattern_dict)
long["工廠"] = long["物料"].map(origin_dict)
long["TBR Premium"] = long["物料"].map(tbr_prem)
long["輪胎類別"] = long["物料"].map(group)
long["品牌"] = long["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")
long["客戶所在地區分區"] = long["買方"].map(cus_area)

In [17]:
long[long["客戶名稱"].isna()]["買方"].unique()

array([], dtype=object)

In [18]:
long["銷售數量"].sum()

2666

In [19]:
long[long["大通路"] != "OE"]["銷售數量"].sum()

2666

In [20]:
long["實際出貨日"] = pd.to_datetime(long["實際出貨日"])

In [21]:
# historical = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")
historical = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")

In [22]:
historical["客戶名稱"] = historical["買方"].map(cuscode_cus)
historical["業務名稱"] = historical["買方"].map(sales_dict)
historical["客戶等級"] = historical["買方"].map(ranking)
historical["業務負責區域"] = historical["買方"].map(sales_area)
historical["業務所屬區域"] = historical["買方"].map(sales_branch)
historical["客戶城市"] = historical["買方"].map(cus_city)
historical["母通路"] = historical["買方"].map(rep_dict)
historical["大通路"] = historical["買方"].map(cuscode_type)
historical["小通路"] = historical["買方"].map(cuscode_channel)
historical["日報通路"] = historical["買方"].map(liwei_dict)
historical["客戶所在地區分區"] = historical["買方"].map(cus_area)

In [23]:
historical["訂單單號"] = historical["訂單單號"].map(lambda x: x.replace(".0", "") if "." in x else x)
historical["訂單項次"] = historical["訂單項次"].map(lambda x: x.replace(".0", "") if "." in x else x)

In [24]:
historical = historical.drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last")

In [25]:
historical = historical[historical["實際出貨日"] < cur_month]

In [26]:
final = pd.concat([historical, long]).drop_duplicates(subset=["實際出貨日", "訂單單號", "訂單項次"], keep="last")
final["實際出貨日"] = pd.to_datetime(final["實際出貨日"], yearfirst=True)

In [27]:
final.to_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet", index=False)
final.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.xlsx", index=False)

In [25]:
final[final["實際出貨日"].between("2023-07-01", "2023-07-31")].groupby(["大通路"])[["銷售數量"]].sum()

,銷售數量
大通路,
AG,156
BSSC,6076
BSSC-FLT,1351
FLT-REP,1146


In [43]:
df = pd.read_csv(r"C:\Users\kc.hsu\Downloads\STANDARD_aqG6x_BFPTTC店家調查_202307180349_64b60bc1717ed.csv")

In [49]:
df[['店家外觀照片 - 儲存位置', '店內及工作區照片 - 儲存位置', '服務車照片 - 儲存位置']] = df[['店家外觀照片 - 儲存位置', '店內及工作區照片 - 儲存位置', '服務車照片 - 儲存位置']].applymap(lambda x: x.split("&")[0])

In [52]:
df.to_csv(r"D:\kc.hsu\OneDrive - Bridgestone\行銷推廣\掛牌店\20230718_FMC問卷回覆.csv", index=False, encoding="utf-8-sig")

In [20]:
df = pd.read_clipboard()

In [22]:
df["COGS"] = df["COGS"].str.strip()

In [23]:
df.to_clipboard()

## 每日庫存整理

In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory_status.xlsx")

In [ ]:
df["類別"] = df["物料"].map(group)

In [ ]:
df = df[~df["類別"].isna()]

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df["花紋"] = df["物料"].map(pattern_dict)
df["規格"] = df["物料"].map(size_dict)
df["工廠"] = df["物料"].map(origin_dict)
df["TBR Premium"] = df["物料"].map(tbr_prem)
df["品牌"] = df["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")
df["收貨日期"] = df["收貨日期"].dt.date

In [ ]:
df = df[["物料", "類別", "花紋", "規格", "工廠", "TBR Premium", "品牌", "可用的庫存", "收貨日期", "總庫存", "入庫庫存", "撿貨數量"]]

In [ ]:
df["撿貨數量"] = df["撿貨數量"] * -1

In [ ]:
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\庫存\inventory_status_處理過.xlsx", index=False)

### 將xlsx批次轉csv或pkl檔

In [ ]:
"ag" in paths[1]

In [ ]:
paths = []
while True:
    answer = input("選擇路徑?(Y/N)").upper()
    if answer == "Y":
        root = tk.Tk()
        root.withdraw()
        file_path = filedialog.askopenfilenames()
        paths += file_path    
    else:
        break

In [ ]:
for path in paths:
    pd.read_excel(path).to_csv(path.split(".")[0] + ".csv", index=False)

In [ ]:
for path in paths:
    pd.read_csv(path).to_pickle(path.split(".")[0] + ".pkl")

In [ ]:
sc_data = pd.concat([DataExtract(path).sc_extract() for path in paths])

In [ ]:
ag_data = pd.concat([DataExtract(path).ag_extract() for path in paths])

In [ ]:
ag_data["訂單日期"] = ag_data["訂單日期"].map(lambda x: x.split(" ")[0] if len(x) > 10 else x)

In [ ]:
sc_data["銷售數量"] = sc_data["銷售數量"].astype(int)

In [ ]:
result = pd.concat([sc_data, ag_data])

In [ ]:
result["訂單項次"] = result["訂單項次"].astype(str).map(lambda x: x.split(".")[0])

In [ ]:
result["銷貨單價"] = result["銷貨單價"].fillna(0).astype(int)

In [ ]:
result["訂單單號"] = result["訂單單號"].astype(str).map(lambda x: x.split(".")[0])

In [ ]:
result.reset_index(drop=True).to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\2022全年銷售\202201-08.pkl")

In [ ]:
long = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")

In [ ]:
long["客戶名稱"] = long["買方"].map(cuscode_cus)
long["母通路"] = long["買方"].map(rep_dict)
long["大通路"] = long["買方"].map(cuscode_type)
long["小通路"] = long["買方"].map(cuscode_channel)
long["日報通路"] = long["買方"].map(liwei_dict)
long["業務名稱"] = long["買方"].map(sales_dict)
long["業務負責區域"] = long["買方"].map(sales_area)
long["業務所屬區域"] = long["買方"].map(sales_branch)
long["客戶登記地址"] = long["買方"].map(cus_address)
long["客戶城市"] = long["買方"].map(cus_city)
long["客戶等級"] = long["買方"].map(ranking)

In [ ]:
long["規格"] = long["物料"].map(size_dict)
long["花紋"] = long["物料"].map(pattern_dict)
long["工廠"] = long["物料"].map(origin_dict)
long["TBR Premium"] = long["物料"].map(tbr_prem)
long["輪胎類別"] = long["物料"].map(group)
long["品牌"] = long["物料"].map(lambda x: "FS" if x.endswith("F") else "BS")

In [ ]:
long

In [ ]:
long.to_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")
long.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.xlsx", index=False)

## 相關係數分析

In [ ]:
test = historical[['物料', '國別', '銷售數量', '銷貨單價', '客戶名稱', '小通路', '業務名稱', '業務負責區域', '業務所屬區域', '客戶城市', '規格', '花紋', '工廠', 'TBR Premium', '輪胎類別', '品牌',
       '客戶等級']]

In [ ]:
pd.get_dummies(test).corr(method="spearman")

## 小徐要的yahoo汽車對比普利司通輪胎規格花紋

In [ ]:
# 整理李濰的POS PSR規格花紋對照表
df = pd.read_clipboard()

In [ ]:
df.columns = df.columns.str.strip()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df["花紋"] = df["花紋"].str.strip()
df["規格"] = df["規格"].str.strip()
df["料號"] = df["料號"].str.strip()

In [ ]:
# 讀取各個廠牌的規格花紋
datas = []
root = tk.Tk()
root.withdraw()
paths = filedialog.askopenfilenames()

In [ ]:
for path in paths:
    if "conti" not in path:
        df = pd.read_excel(path)
        datas.append(df)
    else:
        conti = pd.read_excel(path)

In [ ]:
mi = pd.concat(datas)

In [ ]:
mi = mi[["規格", "花紋"]]

In [ ]:
all_brand = pd.concat([mi, conti])

In [ ]:
result = all_brand.groupby(["規格"])["花紋"].apply(list).reset_index()

In [ ]:
all_brand_list = dict(zip(result["規格"], result["花紋"]))

In [ ]:
yahoo["competitor"] = yahoo["前輪尺寸"].map(all_brand_list)

In [ ]:
yahoo["competitor"].fillna("", inplace=True)

In [ ]:
competitor_table = pd.DataFrame(yahoo["competitor"].to_list())

In [ ]:
competitor_table

In [ ]:
competitor_table.columns = ["Competitor 1", "Competitor 2", "Competitor 3", "Competitor 4", "Competitor 5", "Competitor 6", "Competitor 7", "Competitor 8", "Competitor 9", 
                            "Competitor 10", "Competitor 11", "Competitor 12", "Competitor 13", "Competitor 14", "Competitor 15", "Competitor 16", "Competitor 17", "Competitor 18", "Competitor 19"]

In [ ]:
export_data = pd.concat([yahoo, competitor_table], axis=1)

In [ ]:
export_data.to_excel(r"C:\Users\kc.hsu\Desktop\20221215_yahoo汽車對比競品花紋大全.xlsx", index=False)

In [ ]:
# 使用groupby將所有一樣規格的花紋打包到list
result = df.groupby(["規格"])["料號"].apply(list).reset_index()

In [ ]:
size_table = dict(zip(result["規格"], result["料號"]))

In [ ]:
# 讀取yahoo汽車規格
yahoo = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\Yahoo汽車資訊\20221212_yahoo汽車對比普利司通花紋.xlsx")

In [ ]:
yahoo = pd.read_clipboard()

In [ ]:
yahoo["BS規格"] = yahoo["前輪尺寸"].map(size_table)

In [ ]:
yahoo["BS規格"].fillna("", inplace=True)

In [ ]:
final = pd.concat([yahoo, pd.DataFrame(yahoo["BS規格"].to_list())], axis=1)

In [ ]:
final.columns = [  '年份',   '品牌',   '車系',   '車型',   '價格', '動力型式',  '排氣量', '驅動型式', '車身型式',
         '車重', '前輪尺寸', '後輪尺寸', 'BS規格',      "BPC 1",      "BPC 2",      "BPC 3",      "BPC 4",      "BPC 5",
            "BPC 6",      "BPC 7",      "BPC 8"]

In [ ]:
pattern_dict = dict(zip(df["料號"], df["花紋"]))

In [ ]:
for i, j in enumerate(["BPC 1",      "BPC 2",      "BPC 3",      "BPC 4",      "BPC 5",
            "BPC 6",      "BPC 7",      "BPC 8"], start=1):
    final[i] = final[j].map(pattern_dict)

In [ ]:
final.columns = [   '年份',    '品牌',    '車系',    '車型',    '價格',  '動力型式',   '排氣量',  '驅動型式',
        '車身型式',    '車重',  '前輪尺寸',  '後輪尺寸',  'BS規格', 'BPC 1', 'BPC 2', 'BPC 3',
       'BPC 4', 'BPC 5', 'BPC 6', 'BPC 7', 'BPC 8',       'Pattern 1',       'Pattern 2',       'Pattern 3',
             'Pattern 4',       'Pattern 5',       'Pattern 6',       'Pattern 7',       'Pattern 8']

In [ ]:
export_data = final[['年份',    '品牌',    '車系',    '車型',    '價格',  '動力型式',   '排氣量',  '驅動型式',
        '車身型式',    '車重',  '前輪尺寸',  '後輪尺寸', 'BPC 1', 'Pattern 1', 'BPC 2', 'Pattern 2', 'BPC 3', 'Pattern 3',
       'BPC 4', 'Pattern 4', 'BPC 5', 'Pattern 5', 'BPC 6', 'Pattern 6', 'BPC 7', 'Pattern 7', 'BPC 8', 'Pattern 8']]

In [ ]:
export_data.to_excel(r"C:\Users\kc.hsu\Desktop\20221212_yahoo汽車對比普利司通花紋.xlsx", index=False)

## 歷史記錄

In [ ]:
df = pd.read_pickle(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.pkl")

In [ ]:
result = df[df["花紋"] == "R154"]

In [ ]:
result = result[["實際出貨日", "買方", "物料", "規格", "花紋", "客戶名稱", "銷售數量"]]

In [ ]:
result.to_excel(r"C:\Users\kc.hsu\Desktop\20230201_R154銷售.xlsx", index=False)

## 產品促銷試算

In [ ]:
df = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")

In [ ]:
## AG 8% discount from the list price without tax
## FLT(BSSC and REP all shows the real net with tax)
## read BSSC price with S net
s_price = pd.read_clipboard()

In [ ]:
s_price["A net"] = s_price["A net"].str.replace(",", "").astype(int)

In [ ]:
df = df[(df["實際出貨日"] >= "2023-02-01") & (df["大通路"] != "OE")]

In [ ]:
s_price["描述"] = s_price["規格"] + " " + s_price["花紋"] + " " + s_price["國別"]

In [ ]:
price_dict = dict(zip(s_price["描述"], s_price["A net"]))

In [ ]:
df["描述"] = df["規格"] + " " + df["花紋"] + " " + df["國別"]

In [ ]:
df["S net"] = df["描述"].map(price_dict)

In [ ]:
df["Price"] = df.apply(lambda x: (x["銷貨單價"] / 1.05) * 0.92 if x["大通路"] == "AG" else x["銷貨單價"] / 1.05 if x["大通路"] in ["BSSC-FLT", "FLT-REP"] else x["S net"], axis=1 )

In [ ]:
df["總NET"] = df["銷售數量"] * df["Price"]

In [ ]:
df.groupby(["大通路"])[["銷售數量", "總NET"]].sum().reset_index().to_clipboard()

In [20]:
df = pd.read_parquet(r"D:\kc.hsu\OneDrive - Bridgestone\數據\銷售數據\長期銷售記錄\2017-202208.parquet")

In [22]:
df["實際出貨日"] = pd.to_datetime(df["實際出貨日"])

In [25]:
df.groupby([pd.Grouper(key="實際出貨日", freq="m"), "輪胎類別", "規格", "花紋"])[["銷售數量"]].sum().to_clipboard()

In [27]:
df[df["規格"] == "825R16"]

,訂單日期,實際出貨日,訂單單號,訂單項次,買方,物料,物料說明,國別,銷售數量,銷貨單價,客戶名稱,母通路,大通路,小通路,日報通路,業務名稱,業務負責區域,業務所屬區域,客戶登記地址,客戶城市,規格,花紋,工廠,TBR Premium,輪胎類別,品牌,客戶等級,客戶所在地區分區
4,2017-01-03,2017-01-03,102883527,30,C1C10131,LSR01270,825 R 16 14P R225Z 日,JP,4,5862,廣億輪胎有限公司,REP,BSSC,A,其他,林俊良,五股、桃園,北區,新北市樹林區三俊街26號,新北市,825R16,R225,BSJ,No,LSR,BS,No contract,北北基
12,2017-01-04,2017-01-04,102885423,40,C1C10335,LSR01270,825 R 16 14P R225Z 日,JP,2,5862,文發輪胎行,REP,BSSC,A,其他,林俊良,北北宜,北區,新北市中和區景平路104號1樓,新北市,825R16,R225,BSJ,No,LSR,BS,No contract,北北基
19,2017-01-04,2017-01-04,102885666,10,C1C10038,LSR01270,825 R 16 14P R225Z 日,JP,5,5862,雙十輪胎行,REP,BSSC,A,其他,林俊良,北北宜,北區,新北市板橋區民生路3段54號,新北市,825R16,R225,BSJ,No,LSR,BS,No contract,北北基
26,2017-01-05,2017-01-05,102886570,10,C1C10132,LSR01270,825 R 16 14P R225Z 日,JP,4,5862,長隆輪胎行,REP,BSSC,A,其他,林俊良,五股、桃園,北區,新北市樹林區保安街2段51號,新北市,825R16,R225,BSJ,No,LSR,BS,No contract,北北基
28,2017-01-05,2017-01-05,102886653,10,C1B10336,LSR01270,825 R 16 14P R225Z 日,JP,7,5862,冠軍輪胎行,REP,BSSC,A,其他,林俊良,五股、桃園,北區,桃園市龜山區復興一路367號,桃園市,825R16,R225,BSJ,No,LSR,BS,No contract,桃竹苗
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141096,2023-03-27,2023-03-27,104472660,10,K1K11013,LSR01270,825 R 16 14P R225Z 日,JP,2,6948,榮隆國際有限公司,REP,BSSC,BFP,FMC,張鴻宗,雲嘉南,南區,臺南市善化區小新里小新營３２５號1樓,台南市,825R16,R225,BSJ,No,LSR,BS,A,雲嘉南
141134,2023-03-28,2023-03-28,104473414,10,C1X0C1H1,LSR01270,825 R 16 14P R225Z 日,JP,2,6331,和泰汽車股份有限公司,REP,BSSC,CD,其他,林俊良,北北宜,北區,台北市松江路121號8-14樓,台北市,825R16,R225,BSJ,No,LSR,BS,No contract,北北基
141166,2023-03-29,2023-03-29,104474401,10,N1N1A020,LSR01270,825 R 16 14P R225Z 日,JP,5,7200,志峰輪胎行,REP,BSSC,A,其他,張正平,高屏台東,南區,台東市正氣北路435號,台東縣,825R16,R225,BSJ,No,LSR,BS,No contract,高屏東
141193,2023-03-29,2023-03-31,104474482,10,K1K11013,LSR01270,825 R 16 14P R225Z 日,JP,15,6948,榮隆國際有限公司,REP,BSSC,BFP,FMC,張鴻宗,雲嘉南,南區,臺南市善化區小新里小新營３２５號1樓,台南市,825R16,R225,BSJ,No,LSR,BS,A,雲嘉南
